In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import joblib


ModuleNotFoundError: No module named 'pandas'

In [27]:
df = pd.read_csv('src/DataSet/heart.csv')

df_atualizado = df.copy()




In [ ]:
x = df_atualizado.drop('HeartDisease', axis=1)

y = df_atualizado['HeartDisease']

df_atualizado = pd.get_dummies(df, drop_first=True)



In [14]:
x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(x_train, y_train)


RandomForestClassifier()

In [17]:
y_pred = model.predict(x_test)
print("Acurácia:", accuracy_score(y_test, y_pred))

joblib.dump(model, 'modelo_cardiaco.pkl')

Acurácia: 0.8641304347826086


['modelo_cardiaco.pkl']

In [26]:
scores = cross_val_score(model, x, y, cv=5)
print("Acuracia por rodada (cross_validation):", scores)
print("Media de acuracia (cross_validation):", scores.mean())

joblib.dump(model, 'modelo_cardiaco.pkl')


Acuracia por rodada (cross_validation): [0.86413043 0.82608696 0.83695652 0.83060109 0.74863388]
Media de acuracia (cross_validation): 0.8212817771442147


['modelo_cardiaco.pkl']